In [1]:
import numpy as np
import librosa
import os
import soundfile as sf
from scipy.io import wavfile
from scipy.signal import butter, lfilter, resample
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2
cutoff_frequency = 195 
sampling_rate=4000

In [2]:
from PIL import Image
spec_data=[]

In [3]:
def data(root_directory,final_class_labels):
    for folder_name, subfolders, filenames in os.walk(root_directory):
        # Iterate through audio files in the current folder
        for filename in filenames:
            if filename.lower().endswith(('.png')):
#                 Construct the full path to the audio file
                audio_path = os.path.join(folder_name, filename)
                img=Image.open(audio_path).convert('RGB')
                img=img.resize((128,128))
                img_array = np.asarray(img)
                img_array = img_array / 255
                spec_data.append([img_array,final_class_labels])   

In [4]:
data('/Users/vaishnavikrovvidi/Desktop/ps2/pascalheartds/Dataset_A/Spec-A/Atraining_artifact','artifact')
data('/Users/vaishnavikrovvidi/Desktop/ps2/pascalheartds/Dataset_A/SpecAugment-A/Atraining_artifact','artifact')

In [5]:
data('/Users/vaishnavikrovvidi/Desktop/ps2/pascalheartds/Dataset_A/Spec-A/Atraining_extrahls','extrahls')
data('/Users/vaishnavikrovvidi/Desktop/ps2/pascalheartds/Dataset_A/SpecAugment-A/Atraining_extrahls','extrahls')

In [6]:
data('/Users/vaishnavikrovvidi/Desktop/ps2/pascalheartds/Dataset_A/Spec-A/Atraining_murmur','murmur')
data('/Users/vaishnavikrovvidi/Desktop/ps2/pascalheartds/Dataset_A/SpecAugment-A/Atraining_murmur','murmur')

In [7]:
data('/Users/vaishnavikrovvidi/Desktop/ps2/pascalheartds/Dataset_A/Spec-A/Atraining_normal','normal')
data('/Users/vaishnavikrovvidi/Desktop/ps2/pascalheartds/Dataset_A/SpecAugment-A/Atraining_normal','normal')

In [8]:
import pandas as pd
spec_data=pd.DataFrame(spec_data,columns=['feature','class'])
spec_data.head()

,feature,class
0,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
1,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
2,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
3,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact
4,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,...",artifact


In [9]:
spec_data['class'].value_counts()

class
artifact    720
normal      408
murmur      396
extrahls    222
Name: count, dtype: int64

In [10]:
X=np.array(spec_data['feature'].tolist())
y=np.array(spec_data['class'].tolist())

In [11]:
X.shape

(1746, 128, 128, 3)

In [12]:
#downsampling artifact in dataset-A (drop 50%)

import numpy as np

# Identify the class label you want to downsample
class_label = 'artifact'  # Update with your desired class label

# Separate the data into eliminated and remaining subsets
eliminated_subset_X = X[y == class_label]
eliminated_subset_y = y[y == class_label]
remaining_subset_X = X[y != class_label]
remaining_subset_y = y[y != class_label]

# Randomly select 50% of the data from the remaining subset
num_eliminated_samples = len(eliminated_subset_X)
num_samples_to_select = int(num_eliminated_samples * 0.5)

random_indices = np.random.choice(num_eliminated_samples, size=num_samples_to_select, replace=False)
randomly_selected_X = eliminated_subset_X[random_indices]
randomly_selected_y = eliminated_subset_y[random_indices]

# Combine the eliminated subset and the randomly selected subset
downsampled_X = np.concatenate((remaining_subset_X, randomly_selected_X), axis=0)
downsampled_y = np.concatenate((remaining_subset_y, randomly_selected_y), axis=0)


In [13]:
downsampled_X.shape

(1386, 128, 128, 3)

In [14]:
X=downsampled_X
y=np.array(pd.get_dummies(downsampled_y))

In [15]:
X.shape

(1386, 128, 128, 3)

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
X_train,X_valid,y_train,y_valid=train_test_split(X_train,y_train,test_size=0.15,random_state=42)

In [18]:
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout

# Load the pre-trained ResNet152V2 model
resnet152v2 = ResNet152V2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the pre-trained layers
for layer in resnet152v2.layers:
    layer.trainable = False

# Add custom layers on top of the pre-trained model
flatten_layer = Flatten()(resnet152v2.output)
dense_layer = Dense(256, activation='relu')(flatten_layer)
dropout_layer = Dropout(0.35)(dense_layer)
classification_layer = Dense(4, activation='softmax')(dropout_layer)  # Replace 'num_classes' with the number of classes in your problem

# Create the final model
model = Model(inputs=resnet152v2.input, outputs=classification_layer)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


2023-07-11 12:19:46.128562: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-11 12:19:48.951951: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-11 12:19:48.952341: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 pool1_pad (ZeroPadding2D)      (None, 66, 66, 64)   0           ['conv1_conv[0][0]']         

 vation)                                                                                          
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 32, 32, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 32, 32, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 32, 32, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
          

 conv3_block2_2_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 16, 16, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_out (Add)         (None, 16, 16, 512)  0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_conv[0][0]']    
          

 ization)                                                                                         
                                                                                                  
 conv3_block5_1_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block5_2_pad (ZeroPaddin  (None, 18, 18, 128)  0          ['conv3_block5_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block5_2_conv (Conv2D)   (None, 16, 16, 128)  147456      ['conv3_block5_2_pad[0][0]']     
                                                                                                  
 conv3_blo

...

 conv4_block29_1_bn (BatchNorma  (None, 8, 8, 256)   1024        ['conv4_block29_1_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block29_1_relu (Activati  (None, 8, 8, 256)   0           ['conv4_block29_1_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block29_2_pad (ZeroPaddi  (None, 10, 10, 256)  0          ['conv4_block29_1_relu[0][0]']   
 ng2D)                                                                                            
                                                                                                  
 conv4_block29_2_conv (Conv2D)  (None, 8, 8, 256)    589824      ['conv4_block29_2_pad[0][0]']    
          

                                                                                                  
 conv4_block32_preact_bn (Batch  (None, 8, 8, 1024)  4096        ['conv4_block31_out[0][0]']      
 Normalization)                                                                                   
                                                                                                  
 conv4_block32_preact_relu (Act  (None, 8, 8, 1024)  0           ['conv4_block32_preact_bn[0][0]']
 ivation)                                                                                         
                                                                                                  
 conv4_block32_1_conv (Conv2D)  (None, 8, 8, 256)    262144      ['conv4_block32_preact_relu[0][0]
                                                                 ']                               
                                                                                                  
 conv4_blo

 conv4_block34_2_bn (BatchNorma  (None, 8, 8, 256)   1024        ['conv4_block34_2_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 conv4_block34_2_relu (Activati  (None, 8, 8, 256)   0           ['conv4_block34_2_bn[0][0]']     
 on)                                                                                              
                                                                                                  
 conv4_block34_3_conv (Conv2D)  (None, 8, 8, 1024)   263168      ['conv4_block34_2_relu[0][0]']   
                                                                                                  
 conv4_block34_out (Add)        (None, 8, 8, 1024)   0           ['conv4_block33_out[0][0]',      
                                                                  'conv4_block34_3_conv[0][0]']   
          

                                                                                                  
 conv5_block1_1_bn (BatchNormal  (None, 4, 4, 512)   2048        ['conv5_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block1_1_relu (Activatio  (None, 4, 4, 512)   0           ['conv5_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_2_pad (ZeroPaddin  (None, 6, 6, 512)   0           ['conv5_block1_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv5_blo

 conv5_block3_3_conv (Conv2D)   (None, 4, 4, 2048)   1050624     ['conv5_block3_2_relu[0][0]']    
                                                                                                  
 conv5_block3_out (Add)         (None, 4, 4, 2048)   0           ['conv5_block2_out[0][0]',       
                                                                  'conv5_block3_3_conv[0][0]']    
                                                                                                  
 post_bn (BatchNormalization)   (None, 4, 4, 2048)   8192        ['conv5_block3_out[0][0]']       
                                                                                                  
 post_relu (Activation)         (None, 4, 4, 2048)   0           ['post_bn[0][0]']                
                                                                                                  
 flatten (Flatten)              (None, 32768)        0           ['post_relu[0][0]']              
          

In [19]:
# from tensorflow.keras.optimizers import SGD

# # Compile the model with SGD optimizer
# optimizer = SGD(learning_rate=0.01, momentum=0.9)
# model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint('heart_modelcpu.h5',save_best_only=True,save_weights_only=False,monitor='val_accuracy',model='max')

In [21]:
import time
start_time = time.time()

In [20]:
num_epochs = 30
num_batch_size = 32
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)
total_time = time.time() - start_time
print(total_time)

Epoch 1/30


2023-06-23 11:24:00.595283: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - ETA: 0s - loss: 2.1076 - accuracy: 0.6435

2023-06-23 11:24:38.453577: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - 63s 1s/step - loss: 2.1076 - accuracy: 0.6435 - val_loss: 0.6518 - val_accuracy: 0.7554
Epoch 2/30
35/35 [==============================] - 36s 1s/step - loss: 0.5564 - accuracy: 0.7861 - val_loss: 0.5129 - val_accuracy: 0.7878
Epoch 3/30
35/35 [==============================] - 37s 1s/step - loss: 0.4045 - accuracy: 0.8285 - val_loss: 0.4784 - val_accuracy: 0.8453
Epoch 4/30
35/35 [==============================] - 35s 1s/step - loss: 0.3481 - accuracy: 0.8601 - val_loss: 0.4136 - val_accuracy: 0.8669
Epoch 5/30
35/35 [==============================] - 33s 946ms/step - loss: 0.2791 - accuracy: 0.8926 - val_loss: 0.4251 - val_accuracy: 0.8633
Epoch 6/30
35/35 [==============================] - 33s 957ms/step - loss: 0.2622 - accuracy: 0.8953 - val_loss: 0.5312 - val_accuracy: 0.8273
Epoch 7/30
35/35 [==============================] - 33s 950ms/step - loss: 0.2634 - accuracy: 0.9125 - val_loss: 0.4397 - val_accuracy: 0.8597
Epoch 8/30
35/35 [====

In [32]:
y_pred=model.predict(X_test)

8/8 [==============================] - 10s 812ms/step


In [22]:
num_epochs = 50
num_batch_size = 32
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_valid, y_valid), callbacks=[checkpointer], verbose=1)
total_time = time.time() - start_time
print(total_time)

Epoch 1/50


2023-07-11 12:22:32.614397: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-11 12:22:33.163894: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


30/30 [==============================] - ETA: 0s - loss: 12.1409 - accuracy: 0.5781

2023-07-11 12:22:58.715259: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


30/30 [==============================] - 42s 972ms/step - loss: 12.1409 - accuracy: 0.5781 - val_loss: 4.2064 - val_accuracy: 0.7126
Epoch 2/50
30/30 [==============================] - 18s 592ms/step - loss: 6.5693 - accuracy: 0.6674 - val_loss: 3.1747 - val_accuracy: 0.7784
Epoch 3/50
30/30 [==============================] - 16s 536ms/step - loss: 4.5165 - accuracy: 0.7311 - val_loss: 3.0795 - val_accuracy: 0.7305
Epoch 4/50
30/30 [==============================] - 18s 623ms/step - loss: 2.7782 - accuracy: 0.7566 - val_loss: 2.1218 - val_accuracy: 0.7904
Epoch 5/50
30/30 [==============================] - 16s 542ms/step - loss: 1.9114 - accuracy: 0.8013 - val_loss: 2.5961 - val_accuracy: 0.7665
Epoch 6/50
30/30 [==============================] - 18s 599ms/step - loss: 1.6227 - accuracy: 0.8278 - val_loss: 1.3127 - val_accuracy: 0.8084
Epoch 7/50
30/30 [==============================] - 18s 594ms/step - loss: 1.3094 - accuracy: 0.8236 - val_loss: 1.3440 - val_accuracy: 0.8144
Epoch 8/5

In [23]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

9/9 [==============================] - 8s 819ms/step - loss: 1.4493 - accuracy: 0.8705
Test Loss: 1.4493283033370972
Test Accuracy: 0.8705036044120789


In [9]:
def save_spect_testing(data, sr, filename,i):
    # Extract spectrogram
    spectrogram = librosa.feature.melspectrogram(y=data, sr=sr)

    # Convert to decibels
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    # Plot spectrogram
    plt.figure(figsize=(1.28,1.28))
    librosa.display.specshow(spectrogram_db, sr=sr)
    plt.savefig(("/Users/vaishnavikrovvidi/Desktop/Dataset_A/Test-A/{}_{}.png").format(filename,i), transparent=True)
    plt.close()

In [10]:
from tensorflow import keras
def pred(audio_path, model_path,filename):
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    
    cutoff_frequency = 1000  # Adjust as needed
    target_sampling_rate = sampling_rate // 10

            # Preprocess the audio
    preprocessed_audio = preprocess_audio(audio, cutoff_frequency, sampling_rate, target_sampling_rate)

            # Split the audio into segments
    segments = split_audio(preprocessed_audio, sampling_rate)
    
    model = keras.models.load_model(model_path)
    i=0
    
    for segment in segments:
        save_spect_testing(segment,target_sampling_rate,filename,i)

        img=Image.open(("/Users/vaishnavikrovvidi/Desktop/Dataset_A/Test-A/{}_{}.png").format(filename,i)).convert('RGB')
        img_arr=np.asarray(img)
        img_arr=img_arr/255

        img_arr = img_arr.reshape(1, 128, 128, 3)
        
        prediction = model.predict(img_arr)
        x=np.argmax(prediction)
        confidence = prediction[0, x]
        i=i+1

#         prediction = model.predict(img_arr)
#         x=np.argmax(prediction)

        classes={0:'Artifact', 1:'Extrasystole', 2:'Murmur', 3:'Normal'}
        print(classes[x],confidence)

In [13]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_murmur/201103291548.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_model12.h5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-28 10:38:22.580016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 7s 7s/step
Murmur 0.9531634
1/1 [==============================] - 0s 252ms/step
Normal 0.51359326


In [14]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_murmur/201108222238.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_model12.h5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-28 10:38:38.488519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 4s 4s/step
Murmur 0.99897206
1/1 [==============================] - 0s 205ms/step
Murmur 0.98232883


In [15]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_extrahls/201102070251.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_model12.h5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-28 10:38:54.149786: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 5s 5s/step
Normal 0.9581252
1/1 [==============================] - 0s 335ms/step
Normal 0.977128


In [33]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_normal/201102270940.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_model12.h5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-23 11:54:12.118133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 7s 7s/step
Normal 0.9446786
1/1 [==============================] - 0s 207ms/step
Murmur 0.8099968
1/1 [==============================] - 0s 125ms/step
Normal 0.8875659


In [35]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_artifact/201105280851.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_model12.h5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-23 11:55:10.491452: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 7s 7s/step
Normal 0.62292457
1/1 [==============================] - 1s 578ms/step
Normal 0.99650085
1/1 [==============================] - 0s 124ms/step
Normal 0.7893154


In [6]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_artifact/201105280851.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_resnet_spectro.hdf5'
filename='fgq'
generate_predictions(audio_path, model_path, filename)

Metal device set to: Apple M1


2023-06-30 11:26:40.158169: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-30 11:26:40.159024: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-06-30 11:26:44.960631: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 4s 4s/step
Murmur 0.972317
1/1 [==============================] - 0s 80ms/step
Normal 0.75910866
1/1 [==============================] - 0s 54ms/step
Murmur 0.98461455


In [29]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Extrasystole.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_resnet_spectro.hdf5'
filename='fgq'
generate_predictions(audio_path, model_path, filename)

8


2023-06-30 11:38:17.757704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
Murmur 0.81206006
1/1 [==============================] - 0s 99ms/step
Murmur 0.9939209
1/1 [==============================] - 0s 53ms/step
Normal 0.39981222
1/1 [==============================] - 0s 55ms/step
Murmur 0.55389374
1/1 [==============================] - 0s 54ms/step
Murmur 0.9476171
1/1 [==============================] - 0s 53ms/step
Murmur 0.9557658
1/1 [==============================] - 0s 53ms/step
Normal 0.39981222
1/1 [==============================] - 0s 53ms/step
Murmur 0.8352618


In [28]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_extrahls/201102070251.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_resnet_spectro.hdf5'
filename='fgq'
generate_predictions(audio_path, model_path, filename)

8


2023-06-30 11:38:04.100146: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
Murmur 0.81206006
1/1 [==============================] - 0s 62ms/step
Murmur 0.9939209
1/1 [==============================] - 0s 66ms/step
Normal 0.39981222
1/1 [==============================] - 0s 52ms/step
Murmur 0.55389374
1/1 [==============================] - 0s 101ms/step
Murmur 0.9476171
1/1 [==============================] - 0s 54ms/step
Murmur 0.9557658
1/1 [==============================] - 0s 54ms/step
Normal 0.39981222
1/1 [==============================] - 0s 57ms/step
Murmur 0.8352618


In [10]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_murmur/201108222238.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_resnet_spectro.hdf5'
filename='fgq'
generate_predictions(audio_path, model_path, filename)

2023-06-30 11:27:44.468924: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step
Normal 0.57088315
1/1 [==============================] - 0s 59ms/step
Normal 0.39981222


In [26]:
def split_audio(audio, segment_length):
    num_segments = len(audio) // segment_length
    segments = [audio[i*segment_length:(i+1)*segment_length] for i in range(num_segments)]
    return segments

In [27]:
from tensorflow import keras
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
from PIL import Image

def generate_predictions(audio_path, model_path, filename):
    audio, sampling_rate = librosa.load(audio_path, sr=None)

    cutoff_frequency = 195  # Adjust as needed
    target_sampling_rate = sampling_rate // 10

    # Preprocess the audio
    preprocessed_audio = preprocess_audio(audio, cutoff_frequency, sampling_rate, target_sampling_rate)

    # Split the audio into segments
    segments = split_audio(preprocessed_audio, sampling_rate)
    print(len(segments))

    model = keras.models.load_model(model_path)
    i = 0

    for segment in segments:
        # Extract spectrogram
        spectrogram = librosa.feature.melspectrogram(y=segment, sr=target_sampling_rate)

        # Convert to decibels
        spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

        # Plot spectrogram
        plt.figure(figsize=(1.28, 1.28))
        librosa.display.specshow(spectrogram_db, sr=target_sampling_rate)
        plt.savefig(("/Users/vaishnavikrovvidi/Desktop/Dataset_A/Test-A/{}_{}.png").format(filename, i), transparent=True)
        plt.close()

        img = Image.open(("/Users/vaishnavikrovvidi/Desktop/Dataset_A/Test-A/{}_{}.png").format(filename, i)).convert(
            'RGB')
        img_arr = np.asarray(img)
        img_arr = img_arr / 255

        img_arr = img_arr.reshape(1, 128, 128, 3)

        prediction = model.predict(img_arr)
        x = np.argmax(prediction)
        confidence = prediction[0, x]
        i = i + 1

        classes = {0: 'Artifact', 1: 'Extrasystole', 2: 'Murmur', 3: 'Normal'}
        print(classes[x], confidence)


In [20]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_murmur/201108222238.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_model12.h5'
filename='fgq'
generate_predictions(audio_path, model_path, filename)

2023-06-28 12:03:25.355041: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 6s 6s/step
Murmur 0.99897206
1/1 [==============================] - 0s 238ms/step
Murmur 0.98232883


In [39]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint('heart_model12.hdf5',save_best_only=True,save_weights_only=False,monitor='val_accuracy',model='max')

In [40]:
num_epochs = 30
num_batch_size = 32
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)

Epoch 1/30


2023-06-28 14:44:05.564503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - ETA: 0s - loss: 2.4439 - accuracy: 0.6218

2023-06-28 14:44:35.751684: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - 49s 976ms/step - loss: 2.4439 - accuracy: 0.6218 - val_loss: 0.4849 - val_accuracy: 0.8094
Epoch 2/30
35/35 [==============================] - 23s 659ms/step - loss: 0.4975 - accuracy: 0.8032 - val_loss: 0.4418 - val_accuracy: 0.8381
Epoch 3/30
35/35 [==============================] - 23s 654ms/step - loss: 0.3244 - accuracy: 0.8827 - val_loss: 0.3794 - val_accuracy: 0.8453
Epoch 4/30
35/35 [==============================] - 26s 738ms/step - loss: 0.2504 - accuracy: 0.9070 - val_loss: 0.3586 - val_accuracy: 0.8705
Epoch 5/30
35/35 [==============================] - 24s 696ms/step - loss: 0.2459 - accuracy: 0.9070 - val_loss: 0.3690 - val_accuracy: 0.8813
Epoch 6/30
35/35 [==============================] - 30s 858ms/step - loss: 0.2131 - accuracy: 0.9215 - val_loss: 0.4266 - val_accuracy: 0.8417
Epoch 7/30
35/35 [==============================] - 31s 884ms/step - loss: 0.1761 - accuracy: 0.9332 - val_loss: 0.4052 - val_accuracy: 0.8597
Epoch 8/30

In [20]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_murmur/201108222238.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_model12.hdf5'
filename='fgq'
generate_predictions(audio_path, model_path, filename)

2023-06-28 16:23:07.858942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 6s 6s/step
Murmur 0.9965487
[0.9965487]
1/1 [==============================] - 0s 368ms/step
Murmur 0.98142207
[0.9965487, 0.98142207]
Murmur


In [18]:
def generate_predictions(audio_path, model, filename):
    audio, sampling_rate = librosa.load(audio_path, sr=None)
    cutoff_frequency = 1000  # Adjust as needed
    target_sampling_rate = sampling_rate // 10
    preprocessed_audio = preprocess_audio(audio, cutoff_frequency, sampling_rate, target_sampling_rate)
    segments = split_audio(preprocessed_audio, sampling_rate)
    model = keras.models.load_model(model_path)
    i = 0
    a=[]
    b=[]
    for segment in segments:
        spectrogram = librosa.feature.melspectrogram(y=segment, sr=target_sampling_rate)
        spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
        plt.figure(figsize=(1.28, 1.28))
        librosa.display.specshow(spectrogram_db, sr=target_sampling_rate)
        plt.savefig(("/Users/vaishnavikrovvidi/Desktop/Dataset_A/Test-A/{}_{}.png").format(filename, i), transparent=True)
        plt.close()
        img = Image.open(("/Users/vaishnavikrovvidi/Desktop/Dataset_A/Test-A/{}_{}.png").format(filename, i)).convert('RGB')
        img_arr = np.asarray(img)
        img_arr = img_arr / 255
        img_arr = img_arr.reshape(1, 128, 128, 3)
        prediction = model.predict(img_arr)
        x = np.argmax(prediction)
        confidence = prediction[0, x]
        i = i + 1
        classes = {0: 'Artifact', 1: 'Extrasystole', 2: 'Murmur', 3: 'Normal'}
        print(classes[x], confidence)
        a.append(classes[x])
        b.append(confidence)
        print(b)
    b=b.index(max(b))
    print(a[b])

In [27]:
from tensorflow import keras

In [28]:
import tensorflow as tf
from tensorflow import keras

# Define MobileNetV2 as the base model
base_model = keras.applications.MobileNetV2(
    input_shape=(128, 128, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the layers of the base model
base_model.trainable = False

# Build your custom classification model
model_1 = keras.Sequential([
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')
])

# Print the model summary
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Funct  (None, 4, 4, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense_2 (Dense)             (None, 128)               163968    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 4)                 516       
                                                                 
Total params: 2,422,468
Trainable params: 164,484
Non-tr

In [29]:
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
checkpoint = ModelCheckpoint('heart_mn_cpu.hdf5', save_best_only=True, save_weights_only=False, monitor='val_accuracy', mode='max')


In [39]:
model_1.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[checkpoint])

Epoch 1/50


2023-06-23 11:56:09.839118: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - ETA: 0s - loss: 0.8940 - accuracy: 0.6173

2023-06-23 11:56:18.787979: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


35/35 [==============================] - 16s 343ms/step - loss: 0.8940 - accuracy: 0.6173 - val_loss: 0.6032 - val_accuracy: 0.7338
Epoch 2/50
35/35 [==============================] - 6s 167ms/step - loss: 0.5636 - accuracy: 0.7635 - val_loss: 0.5768 - val_accuracy: 0.7698
Epoch 3/50
35/35 [==============================] - 5s 136ms/step - loss: 0.4666 - accuracy: 0.7915 - val_loss: 0.5351 - val_accuracy: 0.7626
Epoch 4/50
35/35 [==============================] - 6s 178ms/step - loss: 0.4040 - accuracy: 0.8412 - val_loss: 0.4709 - val_accuracy: 0.7950
Epoch 5/50
35/35 [==============================] - 4s 118ms/step - loss: 0.3604 - accuracy: 0.8556 - val_loss: 0.4788 - val_accuracy: 0.7842
Epoch 6/50
35/35 [==============================] - 5s 146ms/step - loss: 0.3280 - accuracy: 0.8791 - val_loss: 0.4651 - val_accuracy: 0.7950
Epoch 7/50
35/35 [==============================] - 4s 123ms/step - loss: 0.3193 - accuracy: 0.8800 - val_loss: 0.4381 - val_accuracy: 0.8453
Epoch 8/50
35/35

In [34]:
import time
start_time = time.time()
model_1.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_valid, y_valid), callbacks=[checkpoint])
total_time = time.time() - start_time
print(total_time)

Epoch 1/50
59/59 [==============================] - ETA: 0s - loss: 0.6652 - accuracy: 0.7389

2023-07-05 15:28:00.889030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


59/59 [==============================] - 12s 201ms/step - loss: 0.6652 - accuracy: 0.7389 - val_loss: 0.4781 - val_accuracy: 0.8108
Epoch 2/50
59/59 [==============================] - 6s 104ms/step - loss: 0.5357 - accuracy: 0.7866 - val_loss: 0.5278 - val_accuracy: 0.7748
Epoch 3/50
59/59 [==============================] - 7s 118ms/step - loss: 0.4404 - accuracy: 0.8163 - val_loss: 0.3859 - val_accuracy: 0.8468
Epoch 4/50
59/59 [==============================] - 6s 100ms/step - loss: 0.3927 - accuracy: 0.8434 - val_loss: 0.4197 - val_accuracy: 0.8168
Epoch 5/50
59/59 [==============================] - 7s 114ms/step - loss: 0.3384 - accuracy: 0.8631 - val_loss: 0.3459 - val_accuracy: 0.8468
Epoch 6/50
59/59 [==============================] - 6s 108ms/step - loss: 0.2966 - accuracy: 0.8838 - val_loss: 0.3123 - val_accuracy: 0.8619
Epoch 7/50
59/59 [==============================] - 6s 107ms/step - loss: 0.2934 - accuracy: 0.8907 - val_loss: 0.3154 - val_accuracy: 0.8799
Epoch 8/50
59/59

In [40]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_murmur/201103291548.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_mn_B_123.hdf5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-23 12:00:59.312934: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step
Murmur 0.6508676
1/1 [==============================] - 0s 62ms/step
Murmur 0.6887553


In [41]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_artifact/201105280851.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_mn_B_123.hdf5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-23 12:01:31.235097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step
Normal 0.9993348
1/1 [==============================] - 0s 164ms/step
Normal 0.7260323
1/1 [==============================] - 0s 60ms/step
Normal 0.9993672


In [42]:
audio_path='/Users/vaishnavikrovvidi/Desktop/Dataset_A/Atraining_normal/201102270940.wav'
model_path='/Users/vaishnavikrovvidi/Heartdisease/heart_mn_B_123.hdf5'
filename='fgq'
pred(audio_path, model_path, filename)

2023-06-23 12:02:08.833270: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step
Normal 0.99217236
1/1 [==============================] - 0s 148ms/step
Normal 0.80713373
1/1 [==============================] - 0s 51ms/step
Normal 0.99786764


In [34]:
import tensorflow as tf

# Assuming you have true_labels and predicted_labels as numpy arrays

accuracy = tf.keras.metrics.categorical_accuracy(y_test, y_pred)
mean_accuracy = tf.reduce_mean(accuracy)

print("Categorical Accuracy:", mean_accuracy.numpy())

Categorical Accuracy: 0.8961039
